In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, count, floor, datediff, current_date
from pyspark.sql import functions as F


In [2]:
# Criar a sessão Spark (criar apenas uma vez)
spark = SparkSession.builder.appName("ClientesETL").getOrCreate()

# Ler o arquivo Parquet com o spark
df_clientes = spark.read.option("datetimeRebaseMode", "LEGACY").parquet('C:\\Users\\Theuzao\\Desktop\\panvel_datalake\\data\\raw\\clientes.parquet')

In [3]:
# Mostrar o schema do dataframe
df_clientes.printSchema()

root
 |-- v_id_cli: string (nullable = true)
 |-- d_dt_nasc: date (nullable = true)
 |-- v_sx_cli: string (nullable = true)
 |-- n_est_cvl: string (nullable = true)



Tratamento dos dados

In [4]:
# Contar os valores faltantes em cada coluna
df_clientes.select([count(when(col(c).isNull(), c)).alias(c) for c in df_clientes.columns]).show()

+--------+---------+--------+---------+
|v_id_cli|d_dt_nasc|v_sx_cli|n_est_cvl|
+--------+---------+--------+---------+
|       0|     4957|   16054|    65269|
+--------+---------+--------+---------+



In [5]:
# Preencher os valores faltantes em `v_sx_cli` com "Não informado representado pela letra N"
df_clientes = df_clientes.fillna({"v_sx_cli": "N"})

In [6]:
# Preencher os valores faltantes em `n_est_cvl` com "Desconhecido representado pela letra 0"
df_clientes = df_clientes.fillna({"n_est_cvl": "0"})


In [7]:
# Preencher os valores faltantes em `d_dt_nasc` com "Desconhecido"
df_clientes = df_clientes.fillna({"d_dt_nasc": "Desconhecido"})

In [8]:
# Criar a coluna `idade` a partir da coluna `d_dt_nasc`
df_clientes = df_clientes.withColumn("idade", 
    floor(datediff(current_date(), col("d_dt_nasc")) / 365))

In [9]:
# Contar os valores faltantes em cada coluna
df_clientes.select([count(when(col(c).isNull(), c)).alias(c) for c in df_clientes.columns]).show()

+--------+---------+--------+---------+-----+
|v_id_cli|d_dt_nasc|v_sx_cli|n_est_cvl|idade|
+--------+---------+--------+---------+-----+
|       0|     4957|       0|        0| 4957|
+--------+---------+--------+---------+-----+



In [10]:
df_clientes.show(5)

+--------------------+----------+--------+---------+-----+
|            v_id_cli| d_dt_nasc|v_sx_cli|n_est_cvl|idade|
+--------------------+----------+--------+---------+-----+
|0000009DB36F622B7639|2003-11-14|       F|        0|   20|
|000000F51C15031D708E|2008-03-23|       F|        0|   16|
|00000F54BE2BBF0E7B13|1986-02-24|       F|        0|   38|
|000013E1FB44D9A9E50F|1984-12-13|       M|        0|   39|
|00001522AD94645C7688|1992-11-10|       F|        2|   31|
+--------------------+----------+--------+---------+-----+
only showing top 5 rows



In [11]:
# Data padrão a ser usado em caso de ausência
default_date = '2001-01-01'  # No formato YYYY-MM-DD


In [12]:
# Substituir datas faltantes pela data padrão
df_clientes = df_clientes.withColumn(
    "d_dt_nasc",
    F.when(F.col("d_dt_nasc").isNull(), default_date).otherwise(F.col("d_dt_nasc"))
)

In [13]:
# Preencher os valores faltantes em `idade` com "Desconhecido representado por 0"
df_clientes = df_clientes.fillna({"idade": "0"})

In [14]:
# Contar os valores faltantes em cada coluna
df_clientes.select([count(when(col(c).isNull(), c)).alias(c) for c in df_clientes.columns]).show()

+--------+---------+--------+---------+-----+
|v_id_cli|d_dt_nasc|v_sx_cli|n_est_cvl|idade|
+--------+---------+--------+---------+-----+
|       0|        0|       0|        0|    0|
+--------+---------+--------+---------+-----+



In [15]:
# Converter todas as colunas para String após a leitura
df_clientes = df_clientes.select([F.col(col_name).cast("string").alias(col_name) for col_name in df_clientes.columns])


In [16]:
# Salvar o DataFrame em formato CSV
df_clientes.write.mode("overwrite").option("header", "true").csv("C:\\Users\\Theuzao\\Desktop\\panvel_datalake\\data\\transformed\\clientes")

In [17]:
# Encerra a sessão Spark
spark.stop()